# Generate Noun Property Ratings (Category-Based)

This notebook rates all nouns on their category-specific properties and saves to JSON/JSONL.

**Input:**
- `categorized_nouns.json` - Nouns organized by category
- `category_properties.json` - Properties for each category

**Output:** 
- `noun_property_ratings.jsonl` - JSONL format (one JSON object per line)
- `noun_property_nested.json` - Nested JSON format (nouns with property objects)

## 1. Setup

In [1]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from tqdm import tqdm
import os
import re
import json

/Users/b/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 2. Configuration

In [2]:
# File paths
CATEGORIES_FILE = "categorized_nouns.json"
PROPERTIES_FILE = "category_properties.json"
OUTPUT_FILE = "noun_property_ratings.jsonl"  # JSONL format (one JSON object per line)

# How often to save progress
SAVE_FREQUENCY = 200

## 3. Load Model

In [3]:
print("Loading model...")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="mps",  # Change to "cuda" for NVIDIA GPU or "cpu" for CPU, mps for mac
    torch_dtype="auto",
    trust_remote_code=False,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

print("Model loaded!")

Loading model...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use mps
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model loaded!


## 4. Load Categories and Properties

In [4]:
# Load categories with nouns
with open(CATEGORIES_FILE, 'r') as f:
    categories_data = json.load(f)

# Load properties for each category
with open(PROPERTIES_FILE, 'r') as f:
    properties_data = json.load(f)

# Extract noun-to-category mapping
noun_to_category = {}
category_to_nouns = categories_data['categories']  # Note: lowercase 'categories'

for category, nouns in category_to_nouns.items():
    for noun in nouns:
        noun_to_category[noun] = category

# Get all unique nouns
all_nouns = list(noun_to_category.keys())

print(f"Loaded {len(all_nouns)} nouns across {len(category_to_nouns)} categories")
print(f"\nCategories:")
for category in category_to_nouns.keys():
    noun_count = len(category_to_nouns[category])
    prop_count = len(properties_data[category])
    print(f"  {category}: {noun_count} nouns, {prop_count} properties")

# Calculate total ratings needed
total_ratings = sum(
    len(nouns) * len(properties_data[category])
    for category, nouns in category_to_nouns.items()
)

print(f"\nTotal ratings needed: {total_ratings:,}")
print(f"Estimated time: {total_ratings * 2 / 3600:.1f} hours (at 2 sec/rating)")

Loaded 1846 nouns across 20 categories

Categories:
  Abstract Concepts, Places, People & Activities: 403 nouns, 10 properties
  Aircraft: 25 nouns, 10 properties
  Birds, Fish, Insects & Other Animals: 95 nouns, 10 properties
  Buildings & Large Structures: 70 nouns, 10 properties
  Cars, Trucks, Boats & Other Motorized Vehicles: 13 nouns, 10 properties
  Domesticated Mammals: 73 nouns, 10 properties
  Fruits: 49 nouns, 10 properties
  Geological Features & Minerals: 96 nouns, 10 properties
  Handheld Objects & Tools: 162 nouns, 10 properties
  Motorcycles: 26 nouns, 10 properties
  Natural Non-Geological Features: 87 nouns, 10 properties
  Non-Motorized Transportation: 11 nouns, 10 properties
  Non-Tree Plants: 13 nouns, 10 properties
  Other Raw Foods: 14 nouns, 10 properties
  Prepared Foods & Meals: 76 nouns, 10 properties
  Reptiles & Amphibians: 58 nouns, 10 properties
  Transportation Infrastructure: 47 nouns, 10 properties
  Trees: 24 nouns, 10 properties
  Wild Mammals: 43 no

## 5. Property Type Detection

In [5]:
def is_boolean_property(property_str):
    """
    Determine if a property is a yes/no question (boolean) or a scale (1-10).
    Boolean properties typically start with question words.
    """
    property_lower = property_str.lower().strip()
    
    # Check if it starts with common question patterns
    question_starters = [
        'is it', 'does it', 'can it', 'has it', 'are they', 
        'do they', 'can they', 'is there', 'does this'
    ]
    
    for starter in question_starters:
        if property_lower.startswith(starter):
            return True
    
    # If it ends with a question mark, it's boolean
    if property_str.strip().endswith('?'):
        return True
    
    # Otherwise, it's a scale property
    return False

## 6. Rating Functions

In [6]:
def rate_noun_boolean(noun, property_str, pipe):
    """
    Rate a noun on a boolean property (true/false question).
    Returns 1 for True/Yes, 0 for False/No, -1 if extraction fails.
    """
    # Create prompt for boolean question
    prompt = f"""Question: For "{noun}", {property_str}

Answer only 'true' or 'false'."""
    
    messages = [{"role": "user", "content": prompt}]
    
    try:
        output = pipe(messages)
        response = output[0]["generated_text"].strip().lower()
        
        # Check for true/false, yes/no patterns
        if 'true' in response or 'yes' in response:
            return 1
        elif 'false' in response or 'no' in response:
            return 0
        
        # Try to find definitive words
        if any(word in response for word in ['definitely', 'certainly', 'absolutely']):
            return 1
        if any(word in response for word in ['not', 'never', 'unlikely']):
            return 0
        
        return -1
    except Exception as e:
        print(f"Error: {noun} - {property_str}: {str(e)}")
        return -1


def rate_noun_scale(noun, property_str, pipe):
    """
    Rate a noun on a scale property (1-10).
    Returns rating 1-10, or -1 if extraction fails.
    """
    # Create prompt for scale rating
    prompt = f"""Rate "{noun}" on the property: {property_str}

Scale: 1 = very low/small/weak, 10 = very high/large/strong

Output only the number 1-10."""
    
    messages = [{"role": "user", "content": prompt}]
    
    try:
        output = pipe(messages)
        response = output[0]["generated_text"].strip()
        
        # Extract first valid number from response
        numbers = re.findall(r'\b([1-9]|10)\b', response)
        if numbers:
            rating = int(numbers[0])
            if 1 <= rating <= 10:
                return rating
        
        return -1
    except Exception as e:
        print(f"Error: {noun} - {property_str}: {str(e)}")
        return -1


def rate_noun_on_property(noun, property_str, pipe):
    """
    Rate a noun on a property. Automatically detects if it's boolean or scale.
    Returns the rating (0/1 for boolean, 1-10 for scale, -1 for failure).
    """
    if is_boolean_property(property_str):
        return rate_noun_boolean(noun, property_str, pipe)
    else:
        return rate_noun_scale(noun, property_str, pipe)

## 7. Test Rating Function

In [7]:
# Quick test with examples
print("Testing...\n")

# Test with a category that has both types of properties
test_category = list(category_to_nouns.keys())[0]
test_noun = category_to_nouns[test_category][0]
test_properties = properties_data[test_category]

print(f"Category: {test_category}")
print(f"Testing noun: {test_noun}\n")

# Test first 3 properties
for i, prop in enumerate(test_properties[:3]):
    is_bool = is_boolean_property(prop)
    rating = rate_noun_on_property(test_noun, prop, pipe)
    prop_type = "boolean" if is_bool else "scale"
    print(f"{i+1}. {prop}")
    print(f"   Type: {prop_type}")
    print(f"   Rating: {rating}")
    print()

Testing...

Category: Abstract Concepts, Places, People & Activities
Testing noun: actor

1. is it a person?
   Type: boolean
   Rating: 1

2. is it a place?
   Type: boolean
   Rating: 0

3. is it an activity?
   Type: boolean
   Rating: 0



## 8. Generate All Ratings

In [8]:
# Check for existing progress
if os.path.exists(OUTPUT_FILE):
    existing_ratings = []
    with open(OUTPUT_FILE, 'r') as f:
        for line in f:
            existing_ratings.append(json.loads(line))
    completed = set((r['noun'], r['property']) for r in existing_ratings)
    print(f"Found {len(existing_ratings)} existing ratings")
else:
    existing_ratings = []
    completed = set()
    print("Starting fresh")

# Create list of all work (noun, category, property)
all_combinations = []
for category, nouns in category_to_nouns.items():
    properties = properties_data[category]
    for noun in nouns:
        for prop in properties:
            all_combinations.append((noun, category, prop))

# Filter out completed work
remaining = [
    combo for combo in all_combinations 
    if (combo[0], combo[2]) not in completed
]

print(f"Remaining: {len(remaining):,} ratings")
print("\nStarting...\n")

Found 12000 existing ratings
Remaining: 6,599 ratings

Starting...



In [9]:
# Main loop
results = []

for i, (noun, category, prop) in enumerate(tqdm(remaining, desc="Rating")):
    # Get rating
    rating = rate_noun_on_property(noun, prop, pipe)
    
    # Determine property type for storage
    is_bool = is_boolean_property(prop)
    
    # Store
    results.append({
        'noun': noun,
        'category': category,
        'property': prop,
        'property_type': 'boolean' if is_bool else 'scale',
        'rating': rating
    })
    
    # Save periodically
    if (i + 1) % SAVE_FREQUENCY == 0:
        # Append new results to file
        with open(OUTPUT_FILE, 'a') as f:
            for result in results:
                f.write(json.dumps(result) + '\n')
        results = []
        print(f"Saved at {i + 1} ratings")

# Final save
if results:
    with open(OUTPUT_FILE, 'a') as f:
        for result in results:
            f.write(json.dumps(result) + '\n')

print(f"\nDone! Saved to {OUTPUT_FILE}")

Rating:   3%|▎         | 200/6599 [02:03<24:46,  4.30it/s]  

Saved at 200 ratings


Rating:   6%|▌         | 401/6599 [02:59<21:52,  4.72it/s]  

Saved at 400 ratings


Rating:   9%|▉         | 601/6599 [04:02<28:14,  3.54it/s]  

...

Saved at 6200 ratings


Rating:  97%|█████████▋| 6400/6599 [1:30:11<04:14,  1.28s/it]

Saved at 6400 ratings


Rating: 100%|██████████| 6599/6599 [1:33:59<00:00,  1.17it/s]


Done! Saved to noun_property_ratings.jsonl


## 9. View Results

In [10]:
# Load and display results
ratings = []
with open(OUTPUT_FILE, 'r') as f:
    for line in f:
        ratings.append(json.loads(line))

print(f"Total ratings: {len(ratings):,}")
failed = sum(1 for r in ratings if r['rating'] == -1)
print(f"Failed ratings: {failed}")
print(f"Success rate: {(len(ratings) - failed) / len(ratings) * 100:.1f}%")

# Count boolean vs scale
boolean_count = sum(1 for r in ratings if r.get('property_type') == 'boolean')
scale_count = sum(1 for r in ratings if r.get('property_type') == 'scale')
print(f"\nBoolean properties: {boolean_count:,}")
print(f"Scale properties: {scale_count:,}")

print("\nFirst 20 ratings:")
for rating in ratings[:20]:
    print(rating)

Total ratings: 18,599
Failed ratings: 1
Success rate: 100.0%

Boolean properties: 13,616
Scale properties: 4,983

First 20 ratings:
{'noun': 'actor', 'category': 'Abstract Concepts, Places, People & Activities', 'property': 'is it a person?', 'property_type': 'boolean', 'rating': 1}
{'noun': 'actor', 'category': 'Abstract Concepts, Places, People & Activities', 'property': 'is it a place?', 'property_type': 'boolean', 'rating': 0}
{'noun': 'actor', 'category': 'Abstract Concepts, Places, People & Activities', 'property': 'is it an activity?', 'property_type': 'boolean', 'rating': 0}
{'noun': 'actor', 'category': 'Abstract Concepts, Places, People & Activities', 'property': 'is it tangible?', 'property_type': 'boolean', 'rating': 0}
{'noun': 'actor', 'category': 'Abstract Concepts, Places, People & Activities', 'property': 'Fame', 'property_type': 'scale', 'rating': 9}
{'noun': 'actor', 'category': 'Abstract Concepts, Places, People & Activities', 'property': 'Physical', 'property_type'

In [11]:
# Create nested format (optional - easier to work with)
# Structure: { noun: { category: str, properties: { property: rating, ... } }, ... }
nested = {}
for rating in ratings:
    if rating['rating'] != -1:  # Skip failed ratings
        noun = rating['noun']
        if noun not in nested:
            nested[noun] = {
                'category': rating['category'],
                'properties': {}
            }
        nested[noun]['properties'][rating['property']] = rating['rating']

with open('noun_property_nested.json', 'w') as f:
    json.dump(nested, f, indent=2)

print("Also saved nested format to: noun_property_nested.json")
print(f"Structure: {len(nested)} nouns with category and properties as nested objects")

Also saved nested format to: noun_property_nested.json
Structure: 1846 nouns with category and properties as nested objects


## 10. Summary Statistics by Category

In [12]:
# Show statistics by category
from collections import defaultdict

category_stats = defaultdict(lambda: {'total': 0, 'failed': 0, 'boolean': 0, 'scale': 0})

for rating in ratings:
    cat = rating['category']
    category_stats[cat]['total'] += 1
    if rating['rating'] == -1:
        category_stats[cat]['failed'] += 1
    if rating.get('property_type') == 'boolean':
        category_stats[cat]['boolean'] += 1
    else:
        category_stats[cat]['scale'] += 1

print("\nStatistics by Category:")
print(f"{'Category':<50} {'Total':<8} {'Failed':<8} {'Boolean':<8} {'Scale':<8} {'Success %'}")
print("="*100)

for cat in sorted(category_stats.keys()):
    stats = category_stats[cat]
    success_rate = (stats['total'] - stats['failed']) / stats['total'] * 100
    print(f"{cat:<50} {stats['total']:<8} {stats['failed']:<8} {stats['boolean']:<8} {stats['scale']:<8} {success_rate:.1f}%")


Statistics by Category:
Category                                           Total    Failed   Boolean  Scale    Success %
Abstract Concepts, Places, People & Activities     4030     0        3224     806      100.0%
Aircraft                                           250      0        150      100      100.0%
Birds, Fish, Insects & Other Animals               950      0        760      190      100.0%
Buildings & Large Structures                       700      0        630      70       100.0%
Cars, Trucks, Boats & Other Motorized Vehicles     130      0        78       52       100.0%
Domesticated Mammals                               730      0        365      365      100.0%
Fruits                                             490      0        294      196      100.0%
Geological Features & Minerals                     960      0        768      192      100.0%
Handheld Objects & Tools                           1620     0        1134     486      100.0%
Motorcycles                     

## 11. Analyze Property Types

In [13]:
# Show which properties are detected as boolean vs scale for each category
print("\nProperty Type Analysis by Category:")
print("="*80)

for category in sorted(properties_data.keys()):
    print(f"\n{category}:")
    properties = properties_data[category]
    
    boolean_props = [p for p in properties if is_boolean_property(p)]
    scale_props = [p for p in properties if not is_boolean_property(p)]
    
    if boolean_props:
        print(f"  Boolean ({len(boolean_props)}):")
        for prop in boolean_props:
            print(f"    - {prop}")
    
    if scale_props:
        print(f"  Scale ({len(scale_props)}):")
        for prop in scale_props:
            print(f"    - {prop}")


Property Type Analysis by Category:

Abstract Concepts, Places, People & Activities:
  Boolean (8):
    - is it a person?
    - is it a place?
    - is it an activity?
    - is it tangible?
    - does it require equipment?
    - is it competitive?
    - is it female?
    - is it indoor?
  Scale (2):
    - Fame
    - Physical

Aircraft:
  Boolean (6):
    - does it have wings?
    - is it for international travel?
    - does it carry passengers?
    - is it military?
    - does it need a runway?
    - can it hover?
  Scale (4):
    - Speed
    - Size
    - Cost
    - Speed

Birds, Fish, Insects & Other Animals:
  Boolean (8):
    - can it fly?
    - is it aquatic?
    - is it a mammal?
    - is it a pet?
    - is it a predator?
    - does it have feathers?
    - is it nocturnal?
    - Is it migratory?
  Scale (2):
    - Size
    - Intelligence

Buildings & Large Structures:
  Boolean (9):
    - is it residential?
    - is it tall?
    - is it a landmark?
    - Is it for work?
    - is 